### Random Acts of Pizza

#### By:  David Skarbrevik, Jayashree Ramen, and Nikki Haas
#### Data Science w207:  Intro to Machine Learning
#### University of California, Berkeley
#### Spring, 2017

In [64]:
#some of the SKLearn features we learnt in class are being depreciated.  They still exist for now, so let's ignore 
#the watnings
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning)

#General Libraries
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json
import pandas as pd
import os
import xgboost as xgb
import datetime
from subprocess import check_output
from matplotlib import pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.mixture import GMM
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MaxAbsScaler

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

#SkLearn construction items
from scipy.sparse.csr import csr_matrix
from scipy.sparse import hstack




%matplotlib inline
import sys
print(sys.version)

3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


#### Customized Features

Will the time of day, week, or year help determine which requests results in pizza?

In [2]:
def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 'morning'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 'midday'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 'evening'
    else: 
        y = 'late_night'
    return y

day_values = ['morning', 'midday', 'evening', 'late_night']
    
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    a['weekday'] = calendar.day_name[datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()].lower()
    a['month'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime("%B").lower()
    a['time_of_day'] = day_time(a)
    return a

a = pd.read_json('train.json')
b = a.apply(human_time, axis=1)
c = pd.read_json('test.json')
d = c.apply(human_time, axis =1)

#### Binarized Model

Take all the categorical data, binarize it, and use PCA to find the most relevant categories

In [3]:
#parse the training set to match the testing set's columns
df = b[list(d.columns)]
tstdf = d[list(d.columns)]

In [4]:
#items to binarize:
# all categorical
# requester_subreddits_at_request
# weekday
# month
# time_of_day

#add-ons:
#make bins for numeric cols to coerece into categorical
# requester_account_age_in_days_at_request
# requester_days_since_first_post_on_raop_at_request
# requester_number_of_comments_at_request
# requester_number_of_comments_in_raop_at_request
# requester_number_of_posts_at_request
# requester_number_of_posts_on_raop_at_request
# requester_number_of_subreddits_at_request
# requester_upvotes_minus_downvotes_at_request
# requester_upvotes_plus_downvotes_at_request

In [5]:
b = pd.get_dummies(df['weekday'])
tstb = pd.get_dummies(tstdf['weekday'])
c = pd.get_dummies(df['time_of_day'])
tstc = pd.get_dummies(tstdf['time_of_day'])
d= pd.get_dummies(df['month'])
tstd= pd.get_dummies(tstdf['month'])
df2 = pd.concat([df,b,c,d], axis=1)
tstdf2 = pd.concat([tstdf,tstb,tstc,tstd], axis=1)

In [6]:
del df2['month']
del tstdf2['month']
del df2['weekday']
del tstdf2['weekday']
del df2['time_of_day']
del tstdf2['time_of_day']

In [7]:

subr = []
for x in df['requester_subreddits_at_request']:
    for i, t in enumerate(x):
            subr.append(t)
subr = set(subr)

tstsubr = []
for x in tstdf['requester_subreddits_at_request']:
    for i, t in enumerate(x):
            tstsubr.append(t)
tstsubr = set(tstsubr)


In [8]:
df4 = pd.get_dummies(df2['requester_subreddits_at_request'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)
tstdf4 = pd.get_dummies(tstdf2['requester_subreddits_at_request'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)



df2 = df2[[ 'request_id','friday',
       'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday',
       'evening', 'late_night', 'midday', 'morning', 'april', 'august',
       'december', 'february', 'january', 'july', 'june', 'march', 'may',
       'november', 'october', 'september']]
tstdf2 = tstdf2[[ 'request_id','friday',
       'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday',
       'evening', 'late_night', 'midday', 'morning', 'april', 'august',
       'december', 'february', 'january', 'july', 'june', 'march', 'may',
       'november', 'october', 'september']]

df5 = pd.concat([a['requester_received_pizza'],df2,df4], axis = 1)
tstdf5 = pd.concat([tstdf2,tstdf4], axis = 1)

In [9]:
### Save the work for ease of use

In [220]:
df5.to_json('binarized_categorical_data.json')
tstdf5.to_json('binarized_categorical_test_data.json')
df6 = pd.read_json('binarized_categorical_data.json')
tstdf6 = pd.read_json('binarized_categorical_test_data.json')

In [221]:
del df6['request_id']
del tstdf6['request_id']

Divide the Binarized Categorical Data - Train into Train and Dev datasets - We will use the Dev dataset for testing the accuracy of the prediction

In [12]:
df6_labels = df6['requester_received_pizza']
del df6['requester_received_pizza']
df6_data = df6

bincat_train_data = df6_data[0:2019]
bincat_train_labels = df6_labels[0:2019]
bincat_test_data = df6_data[2020:]
bincat_test_labels = df6_labels[2020:]

We run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters.

In [13]:
def calc_param(n_features, n_components, covtype, classes):
## Function to compute the total parameters for model
   if covtype == 'full': #Covariance Parameters (n_components, n_features, n_features) Symmetric matrix
        totparam = (n_features*n_components + (n_features*(n_features+1)/2)*n_components)*classes
   elif covtype == 'diag': #Covariance Parameters (n_components, n_features)    
        totparam = (n_features*n_components + n_features*n_components)*classes
   elif covtype == 'spherical': #Covariance Parameters = (n_components,)
        totparam = (n_features*n_components + 1)*classes
   elif covtype == 'tied': #Covariance Parameters (n_features, n_features)  Symmetric matrix
        totparam = (n_features*n_components + (n_features*(n_features+1)/2))*classes
   else:
      print('Invalid Covariance Type %s'%(covtype))
    
   return totparam

def P6():
### STUDENT START ###
    cov_matrix = ['spherical', 'diag', 'tied', 'full'] #Initialize list of covariance types 
    modelnum = 0 #Initialize model number
#Loop through to run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters
    for comp in range(1,8): #Vary the Principal components between 1 and 8
        #Project the train data onto the reduced Principal component subspace
        sklearn_pca = PCA(n_components=comp)
        sklearn_transf = sklearn_pca.fit_transform(bincat_train_data)
        #Divide the projected data into positive and negative class datasets
        positive_train_data = sklearn_transf[bincat_train_labels == True]
        negative_train_data = sklearn_transf[bincat_train_labels == False]
        
        for GMMcomp in range(1,8): #Vary the Gaussian Mixture model components between 1 and 8
           for covtype in cov_matrix: #Loop thru the 4 covariance types for the PCA and GMM components
            totparam = calc_param(comp, GMMcomp, covtype, 2) #Calculate the total parameters for model
            modelnum += 1 #Increment the model number
            if totparam <= 50: #Validate that the total parameters for model is <= 50
            #Train the GMM positive and negative models on the positive and negative datasets respectively
                   gm_mod_pos = GMM(n_components = GMMcomp, covariance_type=covtype)
                   gm_mod_pos.fit(positive_train_data)
                   gm_mod_neg = GMM(n_components = GMMcomp, covariance_type=covtype)
                   gm_mod_neg.fit(negative_train_data)
            #Score the trained model on the test dataset
                   sklearn_test_transf = sklearn_pca.transform(bincat_test_data)
                   pscore = gm_mod_pos.score(sklearn_test_transf)
                   nscore = gm_mod_neg.score(sklearn_test_transf)
           
            #Loop to determine the predicted labels of the test data based on the trained model
                   preds = []
                   for pos, neg in zip(pscore, nscore):
                        if pos > neg: preds.append(1)
                        else: preds.append(0)
            
            #Loop to compare the predicted label with the actual label to determine the no of correct answers
                   correct = 0
                   for pred, label in zip(preds, bincat_test_labels):
                        if pred == label: correct +=1
            #Calculate and print the accuracy of the model
                   print('%d Parameters %d PCA Comp %d GMM Comp %d Cov Matrix %s Accuracy is\t %.2f%%'% (modelnum, totparam, comp, GMMcomp, covtype, (100.0*correct/len(bincat_test_labels))))
            else: #If total parameters > 50 skip
                   print('%d Parameters %d PCA Comp %d GMM Comp %d Cov Matrix %s exceeds 50 params'% (modelnum, totparam, comp, GMMcomp, covtype))

            
### STUDENT END ###

P6()

/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/dep

1 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix spherical Accuracy is	 52.08%
2 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix diag Accuracy is	 52.08%
3 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix tied Accuracy is	 52.08%


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarni

4 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix full Accuracy is	 52.08%
5 Parameters 6 PCA Comp 1 GMM Comp 2 Cov Matrix spherical Accuracy is	 43.07%


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/dep

...

/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-

169 Parameters 16 PCA Comp 7 GMM Comp 1 Cov Matrix spherical Accuracy is	 55.40%
170 Parameters 28 PCA Comp 7 GMM Comp 1 Cov Matrix diag Accuracy is	 61.24%
171 Parameters 70 PCA Comp 7 GMM Comp 1 Cov Matrix tied exceeds 50 params
172 Parameters 70 PCA Comp 7 GMM Comp 1 Cov Matrix full exceeds 50 params


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarni

173 Parameters 30 PCA Comp 7 GMM Comp 2 Cov Matrix spherical Accuracy is	 57.92%
174 Parameters 56 PCA Comp 7 GMM Comp 2 Cov Matrix diag exceeds 50 params
175 Parameters 84 PCA Comp 7 GMM Comp 2 Cov Matrix tied exceeds 50 params
176 Parameters 140 PCA Comp 7 GMM Comp 2 Cov Matrix full exceeds 50 params
177 Parameters 44 PCA Comp 7 GMM Comp 3 Cov Matrix spherical Accuracy is	 59.21%
178 Parameters 84 PCA Comp 7 GMM Comp 3 Cov Matrix diag exceeds 50 params
179 Parameters 98 PCA Comp 7 GMM Comp 3 Cov Matrix tied exceeds 50 params
180 Parameters 210 PCA Comp 7 GMM Comp 3 Cov Matrix full exceeds 50 params
181 Parameters 58 PCA Comp 7 GMM Comp 4 Cov Matrix spherical exceeds 50 params
182 Parameters 112 PCA Comp 7 GMM Comp 4 Cov Matrix diag exceeds 50 params
183 Parameters 112 PCA Comp 7 GMM Comp 4 Cov Matrix tied exceeds 50 params
184 Parameters 280 PCA Comp 7 GMM Comp 4 Cov Matrix full exceeds 50 params
185 Parameters 72 PCA Comp 7 GMM Comp 5 Cov Matrix spherical exceeds 50 params
186 Param

/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-

Best accuracy for this model is :Parameters 8 PCA Comp 2 GMM Comp 1 Cov Matrix diag Accuracy is	 62.82%

In [14]:
clf2 = RandomForestClassifier(n_estimators=12)
clf2.fit(bincat_train_data, bincat_train_labels)

preds2 = clf2.predict(bincat_test_data)

print(classification_report(bincat_test_labels,preds2))

print(metrics.accuracy_score(bincat_test_labels,preds2))

             precision    recall  f1-score   support

      False       0.76      0.95      0.84      1519
       True       0.31      0.06      0.10       501

avg / total       0.64      0.73      0.66      2020

0.733168316832


#### Numeric Model

Take all the numeric data, and build a numeric model

In [15]:
 # setup the training and development data
train_data = pd.read_json('train.json', orient='columns')
train_labels = train_data['requester_received_pizza']
text_data = train_data.iloc[:,6]


# some posts have crazy high vote counts
np.sort(train_data.iloc[:,26])[:-11:-1]

# indices of all numeric variables
all_count_num_vars = [1,2,5,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,26,27]

# only "at time of request" numeric variables
count_num_vars_at_request = [1,2,5,9,11,13,15,17,19,21,24,26]

#for x,y in enumerate(train_data.columns):
#    print(x,y)

#for j in [1,5,6,60,600,659,3000]:
#    for i in all_count_num_vars:
#        print(train_data[j,i])

np.unique(train_data.iloc[:,25])

#train_data['bin_votes'] = bin_votes

np.max(train_data.iloc[:,26])/1000

np.random.seed(1)




# Preparing train/test data
num_data = train_data.iloc[:,all_count_num_vars]
num_labels = train_labels



# randomizing the data
#shuffle = np.random.permutation(np.arange(num_data.shape[0]))
#num_train_data, num_train_labels = num_data[shuffle], num_labels[shuffle]

#num_train_data = num_data[:3000,]
#num_train_labels = num_labels[:3000]

#num_test_data = num_data[3001:,]
#num_test_labels = num_labels[3001:]


num_train_data = num_data[0:2019]
num_train_labels = num_labels[0:2019]
num_test_data = num_data[2020:]
num_test_labels = num_labels[2020:]


# simple logistic regression model


lf = LogisticRegression(C = 100)
lf.fit(num_train_data, num_train_labels)

preds = lf.predict(num_test_data)

print(classification_report(num_test_labels,preds))

print(metrics.accuracy_score(num_test_labels,preds))


             precision    recall  f1-score   support

      False       0.84      0.96      0.89      1532
       True       0.77      0.41      0.54       488

avg / total       0.82      0.83      0.81      2020

0.828712871287


In [16]:
### Let's try a Random Forest Classifier
lf2 = RandomForestClassifier(n_estimators=12)
lf2.fit(num_train_data, num_train_labels)

preds2 = lf2.predict(num_test_data)

print(classification_report(num_test_labels,preds2))

print(metrics.accuracy_score(num_test_labels,preds2))

             precision    recall  f1-score   support

      False       0.87      0.95      0.91      1532
       True       0.79      0.56      0.65       488

avg / total       0.85      0.86      0.85      2020

0.856930693069


In [17]:
#### experiment with xgboost
gbm = xgb.XGBClassifier()
gbm.fit(num_train_data, num_train_labels)
preds3 = gbm.predict(num_test_data)

print(classification_report(num_test_labels,preds3))

print(metrics.accuracy_score(num_test_labels,preds3))


             precision    recall  f1-score   support

      False       0.87      0.96      0.91      1532
       True       0.81      0.55      0.65       488

avg / total       0.85      0.86      0.85      2020

0.858910891089


In [33]:
### Pull in text data
a = pd.read_json('train.json', orient='columns')
data_set = a[['request_text_edit_aware','request_title']]

data_labels = a['requester_received_pizza']

def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = re.sub(r'[^a-zA-Z_0-9_\s]'," ", x).lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()

sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
                ('cv', TfidfVectorizer(analyzer='char_wb', 
                                       #vocabulary=vocabulary,
                                       preprocessor=preprocess,
                                       max_df=0.5, ngram_range=(1,3)))
            ])),

            # Pipeline for standard bag-of-words model for body
            ('get-request', Pipeline([
                ('selector', ItemSelector(key='request_text_edit_aware')),
                ('cv', TfidfVectorizer(analyzer='char_wb', 
                                       #vocabulary=vocabulary,
                                       preprocessor=preprocess,
                                       max_df=0.5, ngram_range=(1,3)))
            ])),
            
           #('get-number', Pipeline([
           #     ('selector', ItemSelector(key='number'))
           # ]))

        ]
    )),

    # Use a SVC classifier on the combined features
    ('nb',LogisticRegression(penalty='l2',C=100)),
])


pipeline.fit(X_train, y_train)
text_pred = pipeline.predict(X_test)
target_names = [ 'No pizza','Got pizza']
print(classification_report(y_test, text_pred, target_names=target_names))


             precision    recall  f1-score   support

   No pizza       0.76      0.81      0.78       762
  Got pizza       0.27      0.22      0.24       248

avg / total       0.64      0.66      0.65      1010



In [34]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class WeekdayExtractor(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        shape = data_dict.shape
        return data_dict[self.key].reshape(shape[0],1)

class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features

class NBWrapper(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, NB):
        self.NB = NB

    def fit(self, x, y=None):
        self.NB.fit(x, y)
        return self

    def transform(self, posts):
        return self.NB.predict(posts)

pipeline4= Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
        ### add in text variables
            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
           #     ('cv', CountVectorizer(analyzer='char_wb', 
           #                           #vocabulary=vocabulary,
           #                            #preprocessor=preprocess,
           #                          max_df=0.5, ngram_range=(1,2))),
                ('tfidf', TfidfVectorizer(min_df=65, 
                                          preprocessor=preprocess,
                                          max_df=0.5, ngram_range=(1,3)
                                         ))
            ])),

            # Pipeline for standard bag-of-words model for body
            #('get-request', Pipeline([
            #    ('selector', ItemSelector(key='weekday')),
            #    ('cv', DictVectorizer( 
            #                           #vocabulary=vocabulary,
            #                           ))
            #])),
            
            ('get-text', Pipeline([
                ('selector', ItemSelector(key='request_text_edit_aware')),
                ('tfidf', TfidfVectorizer(min_df=50,
                                          preprocessor=preprocess,
                                          max_df=0.5, ngram_range=(1,3)
                                         ))
                #('cv', CountVectorizer(analyzer='char_wb', 
                #                       #vocabulary=vocabulary,
                #                       max_df=0.5, ngram_range=(1,3)))
            ]))

        

        ]
    )),

    # Use a SVC classifier on the combined features
    #('estimator',LogisticRegression(penalty='l2',C=25))
    #('estimator', SVC(kernel='linear')) 
    ('estimator', DecisionTreeClassifier())
    #('estimator', RandomForestClassifier())
    #('estimator', MultinomialNB())
    #('estmator', KNeighborsClassifier())
])

pipeline4.fit(X_train, y_train)
text_pred = pipeline4.predict(X_test)
target_names = [ 'No pizza','Got pizza']
print(classification_report(y_test, text_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.77      0.77      0.77       762
  Got pizza       0.30      0.30      0.30       248

avg / total       0.66      0.66      0.66      1010



#### Single Data Type Models

It appears that for a single type of data (categorical, numeric, text), the numeric data with a random forest model performs the best.  The next question is, can we combine model types to make a stronger model?

A data scientist at SAP/Concur suggested that we should do each task separately, then combine the data together for one model, using Logistic Regression.

In [201]:
### data needed:
#binarized categorical data: df6_data 
#numeric data: num_data
#numeric data cannot be used as is.  
#Take num_data and normalize all fields
#text data:  data_set
#text data's pipeline masks some of the data, let's redo the Tf-Idf vectorization

In [202]:
#numeric
cols = ['requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request','requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request']
scl = MaxAbsScaler()
num_norm = pd.DataFrame(index=num_data.index)
for col in cols:
    scl.fit(num_data[col].values.reshape(-1,1))
    c = col+'_scaled'
    num_norm[c] = scl.transform(num_data[col].values.reshape(-1, 1))

In [203]:
#text
tkn_request = TfidfVectorizer(preprocessor=preprocess,stop_words='english')
tkn_title = TfidfVectorizer(preprocessor=preprocess,stop_words='english')
feature_request = tkn_request.fit_transform(data_set['request_text_edit_aware'])
feature_title = tkn_title.fit_transform(data_set['request_title'])

In [204]:
#stack the text features
text_X = hstack((feature_request, feature_title), format='csr')
#text_X = hstack((feature_title), format='csr')

In [205]:
# Transform dense matrices to sparse formats so that they can be concatenated
X = hstack((text_X,
            csr_matrix(df6_data), 
            csr_matrix(num_norm)), 
           format='csr')

In [206]:
y = df6_labels

In [207]:
sss = StratifiedShuffleSplit(test_size=0.33, random_state=1)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [208]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
target_names = [ 'No pizza','Got pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.77      0.93      0.84      1021
  Got pizza       0.33      0.12      0.17       313

avg / total       0.67      0.74      0.69      1334



In [209]:
model1 = LogisticRegression()
model1.fit(X, y)
y_pred = model1.predict(X)
print(classification_report(y, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.91      1.00      0.95      3046
  Got pizza       0.99      0.69      0.81       994

avg / total       0.93      0.92      0.92      4040



In [117]:
#do the same to test data
#binarized data: tstdf6
#numeric data: tstdf_num = tstdf[[num_data.columns]]
#numeric data cannot be used as is, normalize all fields
#text data:  tstdf_text = tstdf[]
#text data's pipeline masks some of the data, let's redo the Tf-Idf vectorization

In [210]:
#numeric

tstdf_num = tstdf[cols]
scl = MaxAbsScaler()
num_norm_test = pd.DataFrame(index=tstdf_num.index)
for col in cols:
    scl.fit(tstdf_num[col].values.reshape(-1,1))
    c = col+'_scaled'
    num_norm_test[c] = scl.transform(tstdf_num[col].values.reshape(-1, 1))

In [211]:
#text
tfeature_request = tkn_request.transform(tstdf['request_text_edit_aware'])
tfeature_title = tkn_title.transform(tstdf['request_title'])

In [223]:
### change the binarized test data to contain the same columns as the train data
#df6_cols = [x for x in df6_data.columns if x in tstdf6.columns]
ntstdf6 = pd.DataFrame(index=tstdf6.index)
for x in list(df6_data.columns):
    if x in tstdf6.columns:
        ntstdf6[x] = tstdf6[x]
    else:
        ntstdf6[x] = 0
ntstdf6.shape
#tstdf6 = tstdf6[df6_cols]

(1631, 7507)

In [224]:
df6_data.shape

(4040, 7507)

In [225]:
X_test = hstack((csr_matrix(tfeature_title), 
            csr_matrix(tfeature_request),
            csr_matrix(ntstdf6), 
            csr_matrix(num_norm_test)), 
           format='csr')

In [226]:
Y_test_pred = model1.predict(X_test)
Y_test_pred


array([ True, False, False, ..., False, False, False], dtype=bool)

In [233]:
test = pd.read_json('test.json')
def inter(x):
    return int(x)
vint = np.vectorize(inter)
pred2 = vint(Y_test_pred)

In [236]:
fin_df = pd.concat([test['request_id'],pd.DataFrame(pred2, columns=['requester_received_pizza'])],axis =1)

In [237]:
fin_df.head()

,request_id,requester_received_pizza
0,t3_i8iy4,1
1,t3_1mfqi0,0
2,t3_lclka,0
3,t3_1jdgdj,0
4,t3_t2qt4,0


In [238]:
fin_df.to_csv('pizza_submission2.csv',sep = ',', header=True, index=False )